<a href="https://colab.research.google.com/github/lavanyasatpute/lavanyasatpute/blob/main/Lavanya_with_VGG19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# prompt: mount google drive

from google.colab import drive
drive.mount('/content/drive')


In [ ]:
# prompt: i give the data from drive to colab i.e imges classification data with folder

import os

# Path to your image data in Google Drive
data_dir = '/content/drive/MyDrive/My Documents/final datset'

# List the folders within the data directory
folders = os.listdir(data_dir)
print(data_dir)


In [ ]:
# prompt: how to give it imges in that directory

import cv2
import numpy as np

image_data = []
labels = []

for folder in folders:
  folder_path = os.path.join(data_dir, folder)
  if os.path.isdir(folder_path):
    image_files = os.listdir(folder_path)
    for image_file in image_files:
      image_path = os.path.join(folder_path, image_file)
      image = cv2.imread(image_path)
      if image is not None:
        # Preprocess the image if needed (e.g., resize, normalize)
        # ...
        image_data.append(image)
        labels.append(folder)  # Use the folder name as the label


In [ ]:
labels[520]

In [ ]:
# prompt: write code for spliting a data into validation and testing set

from sklearn.model_selection import train_test_split

# Assuming 'data' is your dataframe
train_data, temp_data = train_test_split(image_data, test_size=0.2, random_state=42)
val_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42)


In [ ]:
train_label, temp_label = train_test_split(labels, test_size=0.2, random_state=42)
val_label, test_label = train_test_split(temp_data, test_size=0.5, random_state=42)

In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Paths
dataset_path = '/content/drive/MyDrive/My Documents/final datset'
train_path = 'Dataset_splits/train'
valid_path = 'Dataset_splits/valid'
test_path = 'Dataset_splits/test'

# Create directories for train, valid, and test splits
os.makedirs(train_path, exist_ok=True)
os.makedirs(valid_path, exist_ok=True)
os.makedirs(test_path, exist_ok=True)

classes = [d for d in os.listdir(dataset_path) if os.path.isdir(os.path.join(dataset_path, d))]

for cls in classes:
    class_path = os.path.join(dataset_path, cls)
    images = [os.path.join(class_path, f) for f in os.listdir(class_path) if os.path.isfile(os.path.join(class_path, f))]

    # Split images into train, valid, and test sets
    train_imgs, temp_imgs = train_test_split(images, test_size=0.30, random_state=42)  # 75% train, 15% valid + 15% test
    valid_imgs, test_imgs = train_test_split(temp_imgs, test_size=0.50, random_state=42)  # 50% of the remaining 30% = 15% valid, 15% test

    # Create class directories in train, valid, and test splits
    os.makedirs(os.path.join(train_path, cls), exist_ok=True)
    os.makedirs(os.path.join(valid_path, cls), exist_ok=True)
    os.makedirs(os.path.join(test_path, cls), exist_ok=True)

    # Copy images to respective directories
    for img in train_imgs:
        shutil.copy(img, os.path.join(train_path, cls))
    for img in valid_imgs:
        shutil.copy(img, os.path.join(valid_path, cls))
    for img in test_imgs:
        shutil.copy(img, os.path.join(test_path, cls))

print("Dataset splitting complete!")

In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# Define the path to your dataset
train_data_dir = '/content/Dataset_splits/train'
validation_data_dir = '/content/Dataset_splits/valid'
weights_dir = '/content/model_weights/'  # Directory to save weights

# Create the directory if it doesn't exist
if not os.path.exists(weights_dir):
    os.makedirs(weights_dir)

# Define image dimensions and batch size
img_width, img_height = 512, 512
batch_size = 32

# Data augmentation for training set
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

# Rescaling for validation set
test_datagen = ImageDataGenerator(rescale=1./255)

# Load the training data
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)

# Load the validation data
validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)

# Load the VGG19 model without the top classification layer
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False

# Add custom classification layers
x = Flatten()(base_model.output)
x = Dense(256, activation='relu')(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Set up ModelCheckpoint to save the weights every 2 epochs
checkpoint = ModelCheckpoint(
    filepath=os.path.join(weights_dir, 'vgg19_weights_epoch_{epoch:02d}.weights.h5'),
    save_weights_only=True,  # Only save the weights
    save_freq='epoch'    # Save after every epoch
)

# Early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=800,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    callbacks=[checkpoint, early_stopping]
)


In [ ]:
# Define the path to your test dataset
test_data_dir = '/content/Dataset_splits/test'

# Load the test data
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)

# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(
    test_generator,
    steps=test_generator.samples // batch_size
)

# Print the test accuracy
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')
